In [1]:
import numpy as np
from pathlib import Path
from brainrender.scene import Scene
from brainrender.video import VideoMaker
from vedo import Points

# Patch the Scene's close method to handle NoneType plotter
def patched_close(self):
    if hasattr(self, 'plotter') and self.plotter is not None:
        self.plotter.close()

Scene.close = patched_close

# Load data
def load_data():
    import pickle
    file_path = 'test_data.pkl'
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    decodability = data['decodability_scores']
    depth_info = data['depth_info']
    x, y, z = depth_info['x_coordinates'], depth_info['y_coordinates'], depth_info['z_coordinates']
    return x, y, z, decodability

# Update scene
def update_scene(scene, x, y, z, decodability, time_index):
    scene.clear_actors()
    current_decodability = decodability[:, time_index]
    coords = np.vstack((x, y, z)).T
    colors = np.interp(current_decodability, (0, 1), (0, 255))
    points = Points(coords, r=10, c=colors)
    scene.add(points)

# Create video
def create_video(x, y, z, decodability, output_path, num_frames):
    scene = Scene(atlas_name=None)  # Disable atlas for testing
    def make_frame(scene, frame_number, nframes=None, resetcam=False, *args, **kwargs):
        update_scene(scene, x, y, z, decodability, frame_number)

    video_maker = VideoMaker(
        scene=scene,
        save_fld=output_path,
        name="decodability_animation",
        fmt="mp4",
        size="1920x1080",
        make_frame_func=make_frame
    )
    video_maker.make_video(duration=num_frames / 10, fps=10)

# Main
if __name__ == "__main__":
    output_folder = Path("output/videos/")
    output_folder.mkdir(parents=True, exist_ok=True)

    x, y, z, decodability = load_data()

    # Testing: Use only a subset
    test_points = 100
    test_timepoints = 10
    x, y, z = x[:test_points], y[:test_points], z[:test_points]
    decodability = decodability[:test_points, :test_timepoints]

    create_video(x, y, z, decodability, output_folder, test_timepoints)


: 

In [1]:
from brainrender import Scene
from brainrender.actors import Points
import numpy as np
import matplotlib.pyplot as plt
import vedo


In [4]:

import vedo 
from brainrender import Scene
popup_scene = Scene(atlas_name='allen_mouse_50um', title='popup')

popup_scene.add_brain_region('VISp')

from vedo import show                                                                                              
vedo.show(*popup_scene.renderables)

: 

In [ ]:
# Initialize the scene
scene = Scene(title="Random Points in VISp")

# Example coordinates
coordinates = np.random.uniform(5000, 7000, (100, 3))
cmap = plt.cm.viridis
colors = [cmap(val)[:3] for val in np.linspace(0, 1, len(coordinates))]



In [3]:
# Add VISp region and points
scene.add_brain_region("VISp", alpha=0.2)
scene.add(Points(coordinates, colors=colors, radius=50))

# Render and save without displaying
scene.render(interactive=True)  

: 